In [23]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os
from selenium.webdriver.common.by import By

os.chdir(r'C:\Users\user\Documents\geckoDriver') # May have to change this path
driver = webdriver.Firefox()
driver.implicitly_wait(30)

# Download the ODI JSONs file
cricURL = 'https://cricsheet.org/'
driver.get(cricURL)

cricMatches = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[1]/p/a[1]')

cricMatches.click()

odiJSON = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[3]/dl/dd[4]/a[1]')

odiJSON.click()

driver.quit()

In [24]:
from zipfile import ZipFile

# Unzip the Zipped JSON Folder
with ZipFile('C:\\Users\\user\\Downloads\\odis_json.zip', 'r') as f:  # May have to change this path
    f.extractall('C:\\Users\\user\\Downloads\\odis_json')  # May have to change this path

In [21]:
import sqlite3
import json
import os

In [22]:
# Specify the folder path containing the JSON files
folder_path = 'C:\\Users\\user\\Downloads\\odis_json'  # May have to change this path

# Get a list of files in the folder
file_list = os.listdir(folder_path)

In [17]:
# Connect to SQLite database
conn = sqlite3.connect('E:\AK-Interview-2023\Zelus\zelusdatabase.db') # May have to change this path
cursor = conn.cursor()

# Create a table to store the Match Details Data (match results)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS odi_match_details (
        match_date date,
        venue TEXT,
        event TEXT,
        match_number TEXT,
        gender TEXT,
        winner TEXT,
        player_of_match TEXT,
        team1 TEXT,
        team2 TEXT,
        margin_runs INTEGER,
        margin_wickets INTEGER,
        result TEXT
    )
''')

# Create a table to store the Team Details (universe of players)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS odi_team_details (
        team_name TEXT,
        player_name TEXT,
        PRIMARY KEY (team_name, player_name)
    )
''')

# Create a table to store the Ball-2-Ball Innings Data (ball-by-ball innings data)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS odi_ball_to_ball_details (
        match_date DATE,
        venue TEXT,
        event TEXT,
        batting_team TEXT,
        overs INTEGER,
        detailed_overs REAL,
        batter TEXT,
        bowler TEXT,
        non_striker TEXT,
        batter_runs INTEGER,
        extra_runs INTEGER,
        total_runs INTEGER,
        wickets INTEGER,
        byes INTEGER, 
        noballs INTEGER, 
        wides INTEGER
    )
''')

In [18]:
# Iterate over each file in the folder
for file_name in file_list:
    # Check if the file is a JSON file
    if file_name.endswith('.json'):
        # Construct the file path
        file_path = os.path.join(folder_path, file_name)
        
        # Read JSON data from file
        with open(file_path, 'r') as file:
            json_data = file.read()
        
        print(file_path)
        
        # Parse JSON data
        data = json.loads(json_data)
        
        # Extract match information
        match_date = data["info"]["dates"][0]
        match_venue = data["info"]["venue"]
        
        if "event" in data["info"]:
            match_event = data["info"]["event"]["name"]
            if "match_number" in data["info"]["event"]:
                match_number = data["info"]["event"]["match_number"]
            else:
                match_number = data["info"]["match_type_number"]
        else:
            match_event = "None"
        
        
        print( "On " + match_date + ", today's event is " + match_event + " at " + match_venue )

        match_gender = data["info"]["gender"]
        match_team1 = data["info"]["teams"][0]
        match_team2 = data["info"]["teams"][1]
        
        # Check if a ["player_of_match"] is present
        if "missing" in data["info"]:
            for element in data["info"]["missing"]:
                if element == "player_of_match":
                    player_of_match = "None"
        elif "player_of_match" not in data["info"]:
            player_of_match = "None"
        else:
            player_of_match = data["info"]["player_of_match"][0]
        
        # Check if a ["info"]["outcome"]["winner"] is present
        if "outcome" in data["info"] and "winner" in data["info"]["outcome"]:
            match_winner = data["info"]["outcome"]["winner"]
            # Check if the win result is by runs or by wickets
            if "by" in data["info"]["outcome"] and "runs" in data["info"]["outcome"]["by"]:
                match_margin_runs = data["info"]["outcome"]["by"]["runs"]
                match_margin_wickets = 0
            else:
                match_margin_runs = 0
                match_margin_wickets = data["info"]["outcome"]["by"]["wickets"]
            # Check if the result is by D/L from ["info"]["outcome"]["method"] key
            if "method" in data["info"]["outcome"]:
                match_result = data["info"]["outcome"]["method"]
            else:
                match_result = "Completed"
        
        else:
            # Key is not present
            match_winner = "None"
            match_margin_runs = 0
            match_margin_wickets = 0
            player_of_match = "None"
            match_result = data["info"]["outcome"]["result"]
        
        # Insert into the odi_match_details table
        cursor.execute("INSERT INTO odi_match_details (match_date, venue, event, match_number, gender, winner, player_of_match, team1, team2, margin_runs, margin_wickets, result) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (match_date, match_venue, match_event, match_number, match_gender, match_winner, player_of_match, match_team1, match_team2, match_margin_runs, match_margin_wickets, match_result))
        
        # Extract team players
        team_players = {}
        teams = data["info"]["teams"]
        for team in teams:
            players = data["info"]["players"][team]
            team_players[team] = players
        
        # Insert/Replace into the odi_team_details table, regarding Team name and Team players.
        for team, players in team_players.items():
            for player in players:
                cursor.execute("INSERT OR REPLACE INTO odi_team_details (team_name, player_name) VALUES (?, ?)", (team, player))
        
        # Extract ball-by-ball innings data
        innings = data["innings"]
        for inning in innings:
            team_name = inning["team"]
            overs = inning["overs"]
            
            for over in overs:
                over_number = over["over"]
                deliveries = over["deliveries"]
                i = 0
                
                for delivery in deliveries:
                    byes = noballs = wides = wickets = 0
                    batter = delivery["batter"]
                    bowler = delivery["bowler"]
                    non_striker = delivery["non_striker"]
                    runs = delivery["runs"]
                    batter_runs = runs["batter"]
                    extras = runs["extras"]
                    total_runs = runs["total"]
                    if "extras" in delivery:
                        if "legbyes" in delivery["extras"]:
                            byes = delivery["extras"]["legbyes"]
                        elif "byes" in delivery["extras"]:
                            byes = delivery["extras"]["byes"]
                        elif "wides" in delivery["extras"]:
                            wides = delivery["extras"]["wides"]
                        elif "noballs" in delivery["extras"]:
                            noballs = delivery["extras"]["noballs"]
                    if "wickets" in delivery:
                        wickets = wickets+1
                    i = i+1
                    # Insert into the odi_ball_to_ball_details table
                    cursor.execute("INSERT INTO odi_ball_to_ball_details (match_date, venue, event, batting_team, overs, detailed_overs, batter, bowler, non_striker, batter_runs, extra_runs, total_runs, wickets, byes, noballs, wides) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (match_date, match_venue, match_event, team_name, over_number, (str(over_number)+"."+str(i)), batter, bowler, non_striker, batter_runs, extras, total_runs, wickets, byes, noballs, wides))
        print( "Insert completed for " + match_date + ", event is " + match_event + " at " + match_venue )

E:\AK-Interview-2023\Zelus\odis_json\1000887.json
On 2017-01-13, today's event is Pakistan in Australia ODI Series at Brisbane Cricket Ground, Woolloongabba
Insert completed for 2017-01-13, event is Pakistan in Australia ODI Series at Brisbane Cricket Ground, Woolloongabba
E:\AK-Interview-2023\Zelus\odis_json\1000889.json
On 2017-01-15, today's event is Pakistan in Australia ODI Series at Melbourne Cricket Ground
Insert completed for 2017-01-15, event is Pakistan in Australia ODI Series at Melbourne Cricket Ground
E:\AK-Interview-2023\Zelus\odis_json\1000891.json
On 2017-01-19, today's event is Pakistan in Australia ODI Series at Western Australia Cricket Association Ground
Insert completed for 2017-01-19, event is Pakistan in Australia ODI Series at Western Australia Cricket Association Ground
E:\AK-Interview-2023\Zelus\odis_json\1000893.json
On 2017-01-22, today's event is Pakistan in Australia ODI Series at Sydney Cricket Ground
Insert completed for 2017-01-22, event is Pakistan in 

E:\AK-Interview-2023\Zelus\odis_json\1022375.json
On 2017-06-18, today's event is ICC Champions Trophy at Kennington Oval
Insert completed for 2017-06-18, event is ICC Champions Trophy at Kennington Oval
E:\AK-Interview-2023\Zelus\odis_json\1026847.json
On 2016-11-04, today's event is Papua New Guinea in Hong Kong ODI Series at Mission Road Ground, Mong Kok
Insert completed for 2016-11-04, event is Papua New Guinea in Hong Kong ODI Series at Mission Road Ground, Mong Kok
E:\AK-Interview-2023\Zelus\odis_json\1026849.json
On 2016-11-06, today's event is Papua New Guinea in Hong Kong ODI Series at Mission Road Ground, Mong Kok
Insert completed for 2016-11-06, event is Papua New Guinea in Hong Kong ODI Series at Mission Road Ground, Mong Kok
E:\AK-Interview-2023\Zelus\odis_json\1026851.json
On 2016-11-08, today's event is Papua New Guinea in Hong Kong ODI Series at Mission Road Ground, Mong Kok
Insert completed for 2016-11-08, event is Papua New Guinea in Hong Kong ODI Series at Mission Ro

E:\AK-Interview-2023\Zelus\odis_json\1040499.json
On 2017-03-24, today's event is Afghanistan v Ireland ODI Series at Greater Noida Sports Complex Ground
Insert completed for 2017-03-24, event is Afghanistan v Ireland ODI Series at Greater Noida Sports Complex Ground
E:\AK-Interview-2023\Zelus\odis_json\1041523.json
On 2017-01-18, today's event is South Africa Women in Bangladesh ODI Series at Sheikh Kamal International Cricket Stadium
Insert completed for 2017-01-18, event is South Africa Women in Bangladesh ODI Series at Sheikh Kamal International Cricket Stadium
E:\AK-Interview-2023\Zelus\odis_json\1041525.json
On 2017-01-20, today's event is South Africa Women in Bangladesh ODI Series at Sheikh Kamal International Cricket Stadium
Insert completed for 2017-01-20, event is South Africa Women in Bangladesh ODI Series at Sheikh Kamal International Cricket Stadium
E:\AK-Interview-2023\Zelus\odis_json\1043953.json
On 2016-11-18, today's event is ICC Women's Championship at Manuka Oval
In

Insert completed for 2018-01-26, event is England tour of Australia and New Zealand at Adelaide Oval
E:\AK-Interview-2023\Zelus\odis_json\1072314.json
On 2018-01-28, today's event is England tour of Australia and New Zealand at Perth Stadium
Insert completed for 2018-01-28, event is England tour of Australia and New Zealand at Perth Stadium
E:\AK-Interview-2023\Zelus\odis_json\1073401.json
On 2017-02-07, today's event is ICC Women's World Cup Qualifier at P Sara Oval
Insert completed for 2017-02-07, event is ICC Women's World Cup Qualifier at P Sara Oval
E:\AK-Interview-2023\Zelus\odis_json\1073403.json
On 2017-02-07, today's event is ICC Women's World Cup Qualifier at Nondescripts Cricket Club Ground
Insert completed for 2017-02-07, event is ICC Women's World Cup Qualifier at Nondescripts Cricket Club Ground
E:\AK-Interview-2023\Zelus\odis_json\1073405.json
On 2017-02-08, today's event is ICC Women's World Cup Qualifier at Nondescripts Cricket Club Ground
Insert completed for 2017-02-

E:\AK-Interview-2023\Zelus\odis_json\1083448.json
On 2017-04-01, today's event is Bangladesh in Sri Lanka ODI Series at Sinhalese Sports Club Ground
Insert completed for 2017-04-01, event is Bangladesh in Sri Lanka ODI Series at Sinhalese Sports Club Ground
E:\AK-Interview-2023\Zelus\odis_json\1085945.json
On 2017-06-24, today's event is ICC Women's World Cup at County Ground
Insert completed for 2017-06-24, event is ICC Women's World Cup at County Ground
E:\AK-Interview-2023\Zelus\odis_json\1085946.json
On 2017-06-24, today's event is ICC Women's World Cup at County Ground
Insert completed for 2017-06-24, event is ICC Women's World Cup at County Ground
E:\AK-Interview-2023\Zelus\odis_json\1085947.json
On 2017-06-25, today's event is ICC Women's World Cup at Grace Road
Insert completed for 2017-06-25, event is ICC Women's World Cup at Grace Road
E:\AK-Interview-2023\Zelus\odis_json\1085948.json
On 2017-06-26, today's event is ICC Women's World Cup at County Ground
Insert completed for 

...

Insert completed for 2014-11-19, event is South Africa in Australia ODI Series at Manuka Oval
E:\AK-Interview-2023\Zelus\odis_json\754729.json
On 2014-11-21, today's event is South Africa in Australia ODI Series at Melbourne Cricket Ground
Insert completed for 2014-11-21, event is South Africa in Australia ODI Series at Melbourne Cricket Ground
E:\AK-Interview-2023\Zelus\odis_json\754731.json
On 2014-11-23, today's event is South Africa in Australia ODI Series at Sydney Cricket Ground
Insert completed for 2014-11-23, event is South Africa in Australia ODI Series at Sydney Cricket Ground
E:\AK-Interview-2023\Zelus\odis_json\754749.json
On 2015-01-18, today's event is Carlton Mid One-Day International Tri-Series at Melbourne Cricket Ground
Insert completed for 2015-01-18, event is Carlton Mid One-Day International Tri-Series at Melbourne Cricket Ground
E:\AK-Interview-2023\Zelus\odis_json\754751.json
On 2015-01-16, today's event is Carlton Mid One-Day International Tri-Series at Sydney C

Insert completed for 2016-02-09, event is England in South Africa ODI Series at SuperSport Park
E:\AK-Interview-2023\Zelus\odis_json\800475.json
On 2016-02-12, today's event is England in South Africa ODI Series at New Wanderers Stadium
Insert completed for 2016-02-12, event is England in South Africa ODI Series at New Wanderers Stadium
E:\AK-Interview-2023\Zelus\odis_json\800477.json
On 2016-02-14, today's event is England in South Africa ODI Series at Newlands
Insert completed for 2016-02-14, event is England in South Africa ODI Series at Newlands
E:\AK-Interview-2023\Zelus\odis_json\803795.json
On 2014-11-28, today's event is United Arab Emirates v Afghanistan ODI Series at ICC Academy
Insert completed for 2014-11-28, event is United Arab Emirates v Afghanistan ODI Series at ICC Academy
E:\AK-Interview-2023\Zelus\odis_json\803797.json
On 2014-11-30, today's event is United Arab Emirates v Afghanistan ODI Series at ICC Academy
Insert completed for 2014-11-30, event is United Arab Emi

Insert completed for 2015-08-07, event is New Zealand in Zimbabwe ODI Series at Harare Sports Club
E:\AK-Interview-2023\Zelus\odis_json\895797.json
On 2016-02-07, today's event is ICC Women's Championship at Bellerive Oval
Insert completed for 2016-02-07, event is ICC Women's Championship at Bellerive Oval
E:\AK-Interview-2023\Zelus\odis_json\895807.json
On 2016-01-12, today's event is India in Australia ODI Series at Western Australia Cricket Association Ground
Insert completed for 2016-01-12, event is India in Australia ODI Series at Western Australia Cricket Association Ground
E:\AK-Interview-2023\Zelus\odis_json\895809.json
On 2016-01-15, today's event is India in Australia ODI Series at Brisbane Cricket Ground, Woolloongabba
Insert completed for 2016-01-15, event is India in Australia ODI Series at Brisbane Cricket Ground, Woolloongabba
E:\AK-Interview-2023\Zelus\odis_json\895811.json
On 2016-01-17, today's event is India in Australia ODI Series at Melbourne Cricket Ground
Insert 

Insert completed for 2015-11-03, event is ICC Women's Championship at Bert Sutcliffe Oval
E:\AK-Interview-2023\Zelus\odis_json\916647.json
On 2015-11-05, today's event is ICC Women's Championship at Bert Sutcliffe Oval
Insert completed for 2015-11-05, event is ICC Women's Championship at Bert Sutcliffe Oval
E:\AK-Interview-2023\Zelus\odis_json\916661.json
On 2016-02-20, today's event is ICC Women's Championship at Bay Oval
Insert completed for 2016-02-20, event is ICC Women's Championship at Bay Oval
E:\AK-Interview-2023\Zelus\odis_json\916663.json
On 2016-02-22, today's event is ICC Women's Championship at Bay Oval
Insert completed for 2016-02-22, event is ICC Women's Championship at Bay Oval
E:\AK-Interview-2023\Zelus\odis_json\919607.json
On 2015-10-01, today's event is Pakistan in Zimbabwe ODI Series at Harare Sports Club
Insert completed for 2015-10-01, event is Pakistan in Zimbabwe ODI Series at Harare Sports Club
E:\AK-Interview-2023\Zelus\odis_json\919609.json
On 2015-10-03, to

On 2016-10-05, today's event is Australia in South Africa ODI Series at Kingsmead
Insert completed for 2016-10-05, event is Australia in South Africa ODI Series at Kingsmead
E:\AK-Interview-2023\Zelus\odis_json\936141.json
On 2016-10-09, today's event is Australia in South Africa ODI Series at St George's Park
Insert completed for 2016-10-09, event is Australia in South Africa ODI Series at St George's Park
E:\AK-Interview-2023\Zelus\odis_json\936143.json
On 2016-10-12, today's event is Australia in South Africa ODI Series at Newlands
Insert completed for 2016-10-12, event is Australia in South Africa ODI Series at Newlands
E:\AK-Interview-2023\Zelus\odis_json\936159.json
On 2017-01-28, today's event is Sri Lanka in South Africa ODI Series at St George's Park
Insert completed for 2017-01-28, event is Sri Lanka in South Africa ODI Series at St George's Park
E:\AK-Interview-2023\Zelus\odis_json\936161.json
On 2017-02-01, today's event is Sri Lanka in South Africa ODI Series at Kingsmead


In [19]:
conn.commit()

In [20]:
conn.close()